In [67]:
import os, json, datetime, pathlib
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
openAiApiKey = os.getenv("OPENAI_API_KEY")
openAiClient = OpenAI(api_key=openAiApiKey)


We need to store the log files that we are gathering from customers.

In [69]:
if "logsBaseDir" not in globals():
    logsBaseDir = pathlib.Path("logs")
if logsBaseDir.exists() == False:
    logsBaseDir.mkdir(parents=True, exist_ok=True)

if "customerInterestLogPath" not in globals():
    customerInterestLogPath = logsBaseDir / "customer_interest.jsonl"

if "feedbackLogPath" not in globals():
    feedbackLogPath = logsBaseDir / "feedback.jsonl"

# New: ensure pricing_estimates.json exists
if "pricingEstimatesPath" not in globals():
    pricingEstimatesPath = logsBaseDir / "pricing_estimates.json"
    

pricingEstimatesLogPath = pricingEstimatesPath

This tool records a potential customer's interest. It prints a readable line and appends a JSON entry to logs/customer_interest.jsonl 

In [70]:
def recordCustomerInterest(email, name, message):
    validationIssues = []

    if email is None:
        validationIssues.append("email is missing")
    else:
        if isinstance(email, str) == False:
            validationIssues.append("email must be a string")
        else:
            if len(email.strip()) == 0:
                validationIssues.append("email cannot be empty")

    if name is None:
        validationIssues.append("name is missing")
    else:
        if isinstance(name, str) == False:
            validationIssues.append("name must be a string")
        else:
            if len(name.strip()) == 0:
                validationIssues.append("name cannot be empty")

    if message is None:
        validationIssues.append("message is missing")
    else:
        if isinstance(message, str) == False:
            validationIssues.append("message must be a string")
        else:
            if len(message.strip()) == 0:
                validationIssues.append("message cannot be empty")

    if len(validationIssues) > 0:
        return {"status": "error", "issues": validationIssues}

    logEntry = {
        "tool": "record_customer_interest",
        "email": email.strip(),
        "name": name.strip(),
        "message": message.strip()
    }

    print(f"[record_customer_interest]  {name.strip()} <{email.strip()}> | {message.strip()}")


    logDir = customerInterestLogPath.parent
    if logDir.exists() == False:
        logDir.mkdir(parents=True, exist_ok=True)

    try:
        with open(customerInterestLogPath, mode="a", encoding="utf-8") as logFile:
            logFile.write(json.dumps(logEntry, ensure_ascii=False))
            logFile.write("\n")
    except FileNotFoundError as e:
        return {"status": "error", "issues": [str(e)], "path": str(customerInterestLogPath)}

    return {"status": "ok", "saved": True, "path": str(customerInterestLogPath)}


This tool records questions the chatbot could not answer. It prints a readable line and appends a JSON entry to logs/feedback.jsonl.

In [71]:
def recordFeedback(question):
    validationIssues = []

    if question is None:
        validationIssues.append("question is missing")
    else:
        if isinstance(question, str) == False:
            validationIssues.append("question must be a string")
        else:
            if len(question.strip()) == 0:
                validationIssues.append("question cannot be empty")

    if len(validationIssues) > 0:
        return {"status": "error", "issues": validationIssues}

    logEntry = {
        "tool": "record_feedback",
        "question": question.strip()
    }

    print(f"[record_feedback]  {question.strip()}")

    # ensure parent directory exists
    logDir = feedbackLogPath.parent
    if logDir.exists() == False:
        logDir.mkdir(parents=True, exist_ok=True)

    try:
        with open(feedbackLogPath, mode="a", encoding="utf-8") as logFile:
            logFile.write(json.dumps(logEntry, ensure_ascii=False))
            logFile.write("\n")
    except FileNotFoundError as e:
        return {"status": "error", "issues": [str(e)], "path": str(feedbackLogPath)}

    return {"status": "ok", "saved": True, "path": str(feedbackLogPath)}


In [72]:
def calculateCandyPrice(homeValue, mode):
    validationIssues = []

    if homeValue is None:
        validationIssues.append("home_value is missing")
    else:
        if isinstance(homeValue, (int, float)) == False:
            validationIssues.append("home_value must be a number")
        else:
            if homeValue <= 0:
                validationIssues.append("home_value must be positive")

    if mode is None:
        validationIssues.append("mode is missing")
    else:
        if isinstance(mode, str) == False:
            validationIssues.append("mode must be a string")
        else:
            allowed = ["transform", "sell"]
            if mode not in allowed:
                validationIssues.append("mode must be 'transform' or 'sell'")

    if len(validationIssues) > 0:
        return {"status": "error", "issues": validationIssues}

    result = {}
    if mode == "transform":
        transformFee = 0.51 * float(homeValue)
        result = {
            "mode": "transform",
            "home_value": float(homeValue),
            "customer_pays": round(transformFee, 2),
            "note": "Transform and return the same home as a candy masterpiece (51% of value)."
        }
    else:
        if mode == "sell":
            payout = 0.85 * float(homeValue)
            result = {
                "mode": "sell",
                "home_value": float(homeValue),
                "payout_to_customer": round(payout, 2),
                "note": "Company buys at market value minus 15% (customer receives 85%)."
            }

    logDir = pricingEstimatesLogPath.parent
    if logDir.exists() == False:
        logDir.mkdir(parents=True, exist_ok=True)

    print(f"[calculate_candy_price] mode={mode} | home_value={float(homeValue)} | result={result}")

    with open(pricingEstimatesLogPath, mode="a", encoding="utf-8") as logFile:
        logFile.write(json.dumps({"tool": "calculate_candy_price", "result": result}, ensure_ascii=False))
        logFile.write("\n")

    return {"status": "ok", "quote": result, "path": str(pricingEstimatesLogPath)}


In [73]:

toolDefinitions = [
    {
        "type": "function", 
        "function": {
            "name": "record_customer_interest", 
            "description": (
                "Capture customer interest details when the user shares contact information or asks to be contacted. "
                "Use this when the user provides an email and a name, plus any message or notes."
            ),
            "parameters": { 
                "type": "object",
                "properties": {
                    "email": {
                        "type": "string",
                        "description": "Customer email address (unvalidated string; model should ensure it looks like an email)."
                    },
                    "name": {
                        "type": "string",
                        "description": "Customer's full name or the name they prefer to be called."
                    },
                    "message": {
                        "type": "string",
                        "description": "Short note about their needs, context, or interest."
                    }
                },
                "required": ["email", "name", "message"],  
                "additionalProperties": False  
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "record_feedback",
            "description": (
                "Log a question the assistant could not answer or not in your business scope."
                "Use this when the assistant is uncertain or lacks the knowledge to respond accurately."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The user's question the assistant could not answer."
                    }
                },
                "required": ["question"],
                "additionalProperties": False
            }
        }
    }, 
    {
    "type": "function",
    "function": {
        "name": "calculate_candy_price",
        "description": (
            "Compute an approximate price for the service based on home value. "
            "mode='transform' means customer pays 51% to transform and keep the home. "
            "mode='sell' means company buys at market minus 15% (customer receives 85%)."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "home_value": {
                    "type": "number",
                    "description": "Estimated current market value of the home."
                },
                "mode": {
                    "type": "string",
                    "enum": ["transform", "sell"],
                    "description": "Choose 'transform' or 'sell'."
                }
            },
            "required": ["home_value", "mode"],
            "additionalProperties": False
        }
    }
    }
]


Executes the correct local function by name and returns a dict response.
    toolName: string (e.g., "record_customer_interest" or "record_feedback")
    toolArgs: dict of arguments exactly as provided by the model.

In [74]:
def executeToolCall(toolName, toolArgs):
    if isinstance(toolArgs, dict) == False:
        return {"status": "error", "issues": ["toolArgs must be a dictionary"]}

    if toolName == "record_customer_interest":
        emailArg = toolArgs.get("email", None)
        nameArg = toolArgs.get("name", None)
        messageArg = toolArgs.get("message", None)
        toolResult = recordCustomerInterest(emailArg, nameArg, messageArg)
        return toolResult
    else:
        if toolName == "record_feedback":
            questionArg = toolArgs.get("question", None)
            toolResult = recordFeedback(questionArg)
            return toolResult
        else:
            if toolName == "calculate_candy_price":
                homeValueArg = toolArgs.get("home_value", None)
                modeArg = toolArgs.get("mode", None)
                toolResult = calculateCandyPrice(homeValueArg, modeArg)
                return toolResult

    return {"status": "error", "issues": [f"unknown tool: {toolName}"]}


In [75]:
import PyPDF2
from pathlib import Path


summaryFilePath = Path("business_summary.txt")  
pdfFilePath = Path("about_business.pdf")
pdfText = ""
summaryText = "" 


with open(summaryFilePath, mode="r", encoding="utf-8") as summaryFile:
    businessSummaryText = summaryFile.read()  

print(businessSummaryText)

"The House of Candy" is a whimsical real-estate and confectionery company that transforms ordinary homes into fully edible candy houses using its signature “dip-dip” chocolate cauldron process. Customers can either sell their homes at market cost minus 15% for conversion and resale, or pay 51% of their home’s value to have it transformed and returned as a candy masterpiece. Founded by Kareem Hassani, the wizard behind the magic cauldron, the company combines real estate, art, and sweetness—offering buyers the chance to live in, sell, or even taste their dream candy home.


In [76]:

with open(pdfFilePath, mode="rb") as pdfStream:
    pdfReader = PyPDF2.PdfReader(pdfStream)  #
    extractedPdfText = ""  


    for pageIndex in range(0, len(pdfReader.pages)):
        pageObject = pdfReader.pages[pageIndex] 
        pageText = pageObject.extract_text()   
        if pageText is None:

            extractedPdfText += f"\n[Page {pageIndex+1} has nothing to get from]\n"
        else:
            
            extractedPdfText += f"\nPAGE {pageIndex+1}\n"
            extractedPdfText += pageText

print(extractedPdfText)


PAGE 1
BUSINESS  NAME: “THE HOUSE OF CANDY ” 
 
MISSION : To conver t people’s houses fully into cand y. For a fee, everything in the house will 
turn into cand y, the doors , walls, couches , EVERYTHING living or not inside the house . 
Customers will be able to live in (or eat ) their houses made now completely  of sweets. We 
also function as a real -estate company , offering to put people’s candy-converted houses 
on the market . We also function as a real -estate company, offering to put people’s candy -
converted houses on the market, selling  them as exclusive edible real -estate pieces or 
deconstructing them into collectible candy furniture for resale. Every property we list is a 
fully transformed, sugar -crafted masterpiece - a blend of real estate and confectionery art - 
allowing buyers to literally own, display, or even taste their dream home.  “We make the 
impossible sugary and joyful by using our trademark “dip -dip” transformation process. ” 
SERVICES OFFERED : We ca

Joins a list of text parts with a chosen separator while skipping Nones. Returns an empty string if there is nothing to join.

In [77]:
from pathlib import Path
import PyPDF2

if "businessSummaryText" not in globals():
    summaryFilePath = Path("business_summary.txt")
    businessSummaryText = ""
    if summaryFilePath.exists() == True:
        with open(summaryFilePath, mode="r", encoding="utf-8") as summaryFile:
            businessSummaryText = summaryFile.read()

if "extractedPdfText" not in globals():
    pdfFilePath = Path("about_business.pdf")
    extractedPdfText = ""
    if pdfFilePath.exists() == True:
        with open(pdfFilePath, mode="rb") as pdfStream:
            pdfReader = PyPDF2.PdfReader(pdfStream)
            pageCount = len(pdfReader.pages)
            pageIndex = 0
            while pageIndex < pageCount:
                pageObject = pdfReader.pages[pageIndex]
                pageText = pageObject.extract_text()
                if pageText is None:
                    extractedPdfText += f"\n[Page {pageIndex+1} has nothing to get from]\n"
                else:
                    extractedPdfText += f"\nPAGE {pageIndex+1}\n"
                    extractedPdfText += pageText
                pageIndex = pageIndex + 1


systemPrompt = (
    "You are the whimsical candy concierge of 'The House of Candy': a Willy Wonka-like guide. "
    "Be playful and vivid, but stay clear and accurate. Never invent facts.\n\n"
    "Operating Rules:\n"
    "1) Use ONLY the business knowledge below (summary + PDF) for facts.\n"
    "2) If you are uncertain or missing details, CALL the tool 'record_feedback' with the exact user question, "
    "   then briefly say a chocolatier will follow up.\n"
    "3) If a user shows interest or shares contact info, politely confirm name and email. "
    "   Then CALL 'record_customer_interest' with email, name, and a short note.\n"
    "4) Encourage interested visitors to leave contact info for quotes, tours, and callbacks.\n"
    "5) Keep tone warm, kind, and concise.\n\n"
    "6) If, across the conversation, you have EMAIL + NAME + MESSAGE, CALL 'record_customer_interest' exactly once in the current turn (it does not have to be one user message). Do this even if you are also calling pricing/timeline tools."
    "7) IF something is outside the scope of your capabilities (see the source of truth below) and not related to the business/not in our services execute the record_feedback. Then express your regret that you cant do said unavailable task. "
    "Business Knowledge (source of truth) — Summary:\n"
    "------------------------------------------------------------\n"
    f"{businessSummaryText}\n"
    "------------------------------------------------------------\n\n"
    "Business Knowledge (source of truth) — PDF Extract:\n"
    "------------------------------------------------------------\n"
    f"{extractedPdfText}\n"
    "------------------------------------------------------------"
)


In [78]:
def buildInitialMessages(systemPromptText):
    if isinstance(systemPromptText, str) == False:
        systemPromptText = "You are the assistant for The House of Candy."
    messages = []
    systemMessage = {"role": "system", "content": systemPromptText}
    messages.append(systemMessage)
    assistantGreeting = {
        "role": "assistant",
        "content": (
            "Welcome to The House of Candy! I'm your candy concierge—ask me about our dip-dip cauldron, "
            "pricing, or timelines. If you'd like a quote, I can collect your name and email to start the magic."
        )
    }
    messages.append(assistantGreeting)
    return messages

messages = buildInitialMessages(systemPrompt)


In [79]:
import json
from openai import OpenAI

def runSingleTurn(userInputText, messagesList, toolSpecs, modelName="gpt-4o-mini"):
    if isinstance(userInputText, str) == False:
        raise ValueError("userInputText must be a string")
    if isinstance(messagesList, list) == False:
        raise ValueError("messagesList must be a list")
    if isinstance(toolSpecs, list) == False:
        raise ValueError("toolSpecs must be a list")

    userMessage = {"role": "user", "content": userInputText}
    messagesList.append(userMessage)

    firstResponse = openAiClient.chat.completions.create(
        model=modelName,
        messages=messagesList,
        tools=toolSpecs,
        tool_choice="auto",
        temperature=0.4
    )

    if len(firstResponse.choices) == 0:
        fallbackText = "Apologies, I could not conjure a reply right now."
        messagesList.append({"role": "assistant", "content": fallbackText})
        return fallbackText, messagesList

    topChoice = firstResponse.choices[0]
    toolCalls = None
    if hasattr(topChoice.message, "tool_calls"):
        toolCalls = topChoice.message.tool_calls

    if toolCalls is not None:
        if len(toolCalls) > 0:
            callIndex = 0
            while callIndex < len(toolCalls):
                singleToolCall = toolCalls[callIndex]
                toolName = singleToolCall.function.name
                toolArgsJson = singleToolCall.function.arguments
                parsedArgs = {}
                if isinstance(toolArgsJson, str) == True:
                    parsedArgs = json.loads(toolArgsJson)
                toolResult = executeToolCall(toolName, parsedArgs)
                toolMessage = {
                    "role": "tool",
                    "tool_call_id": singleToolCall.id,
                    "name": toolName,
                    "content": json.dumps(toolResult, ensure_ascii=False)
                }
                messagesList.append(toolMessage)
                callIndex = callIndex + 1

            secondResponse = openAiClient.chat.completions.create(
                model=modelName,
                messages=messagesList,
                temperature=0.4
            )

            if len(secondResponse.choices) > 0:
                finalText = secondResponse.choices[0].message.content
            else:
                finalText = "Your request was handled, but I could not produce a final message."

            messagesList.append({"role": "assistant", "content": finalText})
            return finalText, messagesList

    assistantText = topChoice.message.content
    if isinstance(assistantText, str) == False:
        assistantText = "I have a response, but it arrived in an unexpected format."
    messagesList.append({"role": "assistant", "content": assistantText})
    return assistantText, messagesList

In [80]:
def agentRespond(userInputText, messagesList, toolSpecs, modelName="gpt-4o-mini", temperature=0.4):
    if isinstance(userInputText, str) == False:
        raise ValueError("userInputText must be a string")
    if isinstance(messagesList, list) == False:
        raise ValueError("messagesList must be a list")
    if isinstance(toolSpecs, list) == False:
        raise ValueError("toolSpecs must be a list")

    userMessage = {"role": "user", "content": userInputText}
    messagesList.append(userMessage)

    firstResponse = openAiClient.chat.completions.create(
        model=modelName,
        messages=messagesList,
        tools=toolSpecs,
        tool_choice="auto",
        temperature=temperature
    )

    if len(firstResponse.choices) == 0:
        fallbackText = "I could not produce a reply right now."
        messagesList.append({"role": "assistant", "content": fallbackText})
        return fallbackText, messagesList

    topChoice = firstResponse.choices[0]

    toolCalls = None
    if hasattr(topChoice.message, "tool_calls"):
        toolCalls = topChoice.message.tool_calls

    # ----- Tool path -----
    if toolCalls is not None:
        if len(toolCalls) > 0:
            # 1) append the assistant message that contains the tool_calls
            assistantToolCallMessage = {
                "role": "assistant",
                "content": topChoice.message.content if isinstance(topChoice.message.content, str) else "",
                "tool_calls": []
            }

            callIndex = 0
            while callIndex < len(toolCalls):
                singleToolCall = toolCalls[callIndex]
                oneToolCallDict = {
                    "id": singleToolCall.id,
                    "type": "function",
                    "function": {
                        "name": singleToolCall.function.name,
                        "arguments": singleToolCall.function.arguments
                    }
                }
                assistantToolCallMessage["tool_calls"].append(oneToolCallDict)
                callIndex = callIndex + 1

            messagesList.append(assistantToolCallMessage)

            # 2) execute each tool and append its tool result message
            callIndex = 0
            while callIndex < len(toolCalls):
                singleToolCall = toolCalls[callIndex]
                toolName = singleToolCall.function.name
                toolArgsJson = singleToolCall.function.arguments

                parsedArgs = {}
                if isinstance(toolArgsJson, str) == True:
                    parsedArgs = json.loads(toolArgsJson)

                toolResult = executeToolCall(toolName, parsedArgs)

                toolMessage = {
                    "role": "tool",
                    "tool_call_id": singleToolCall.id,
                    "name": toolName,
                    "content": json.dumps(toolResult, ensure_ascii=False)
                }
                messagesList.append(toolMessage)

                callIndex = callIndex + 1

            # 3) finalize
            secondResponse = openAiClient.chat.completions.create(
                model=modelName,
                messages=messagesList,
                temperature=temperature
            )

            if len(secondResponse.choices) > 0:
                finalText = secondResponse.choices[0].message.content
            else:
                finalText = "Your request was handled, but a final message was not produced."

            messagesList.append({"role": "assistant", "content": finalText})
            return finalText, messagesList

    # ----- No-tool path -----
    assistantText = topChoice.message.content
    if isinstance(assistantText, str) == False:
        assistantText = "I have a response, but it arrived in an unexpected format."
    messagesList.append({"role": "assistant", "content": assistantText})
    return assistantText, messagesList


In [81]:
messages = buildInitialMessages(systemPrompt)

demoInputLead = (
    "Hi! I'm Kareem, email is kareem@example.com. I want a dip-dip conversion quote. My house's cost is around 10000 dollars all together  "
    "for a 120 sqm house in Hamra next month."
)
finalReplyLead, messages = agentRespond(demoInputLead, messages, toolDefinitions)
print("\n--- Assistant ---\n" + finalReplyLead)


[calculate_candy_price] mode=transform | home_value=10000.0 | result={'mode': 'transform', 'home_value': 10000.0, 'customer_pays': 5100.0, 'note': 'Transform and return the same home as a candy masterpiece (51% of value).'}
[record_customer_interest]  Kareem <kareem@example.com> | Requesting a dip-dip conversion quote for a house costing around $10,000.

--- Assistant ---
Hello Kareem! Thank you for your interest in our magical dip-dip conversion! For your delightful house valued at $10,000, the fee to transform and return it as a candy masterpiece would be **$5,100** (51% of the value).

If you have any more questions or need further assistance, feel free to ask! 🍭✨


In [82]:
%pip install -U pip
%pip install gradio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [84]:


import gradio as gr

def chatWithTools(userText, history):
    if isinstance(history, list) == False:
        history = []

    messagesList = []

    systemMessage = {"role": "system", "content": systemPrompt}
    messagesList.append(systemMessage)

    index = 0
    while index < len(history):
        historyItem = history[index]
        if isinstance(historyItem, dict) == True:
            roleValue = historyItem.get("role", None)
            contentValue = historyItem.get("content", None)

            if isinstance(roleValue, str) == True:
                if isinstance(contentValue, str) == True:
                    if roleValue == "user" or roleValue == "assistant":
                        messagesList.append({"role": roleValue, "content": contentValue})
        index = index + 1

    finalText, _ = agentRespond(userText, messagesList, toolDefinitions)
    return finalText

demo = gr.ChatInterface(
    fn=chatWithTools,
    title="The House of Candy — Wonka Concierge",
    type="messages"
)


demo.launch()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


[record_feedback]  Can I transform my house into pure sand?
